# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd  
import numpy as np
# your code here
# data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

# define the column names
columns = ['target','id','date','flag','user','text']

# read the CSV file with the specified column names
data = pd.read_csv('training.1600000.processed.noemoticon.csv', 
                   header=None, names=columns, encoding='ISO-8859-1', 
                   index_col=False)
# Extracting a random sample of 20000
subset = data.sample(n=20000, random_state=42)

## Reset the column names to match the original dataframe
subset.columns = data.columns

display(subset.head(),subset.shape)

,target,id,date,flag,user,text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


(20000, 6)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [2]:
# your code here

In [3]:
import re

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    # Convert to lowercase
    s = s.lower()

    # Remove HTML tags
    s = re.sub(r'<.*?>', '', s)

    # Remove JavaScript
    s = re.sub(r'<script\b[^<]*(?:(?!<\/script>)<[^<]*)*<\/script>', '', s)

    # Remove CSS
    s = re.sub(r'<style\b[^<]*(?:(?!<\/style>)<[^<]*)*<\/style>', '', s)

    # Remove URLs
    s = re.sub(r'http\S+', '', s)
    s = re.sub(r'www\S+', '', s)

    # Remove numbers ?
    s = re.sub(r'\d+', ' ', s)

    # Remove special characters
    s = re.sub(r'[^\w\s]', ' ', s)

    return s

In [4]:
subset['text_processed'] = subset['text'].apply(clean_up)

In [5]:
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(s)
    tokens = [word.lower() for word in tokens if word.isalnum()]
    return tokens

In [6]:
subset['text_processed'] = subset['text_processed'].apply(tokenize)

In [7]:
subset['text_processed']

541200              [chrishasboobs, ahhh, i, hope, your, ok]
750        [misstoriblack, cool, i, have, no, tweet, apps...
766711     [tiannachaos, i, know, just, family, drama, it...
285055     [school, email, won, t, open, and, i, have, ge...
705995                             [upper, airways, problem]
                                 ...                        
1178000    [loving, the, weather, although, we, re, not, ...
667014           [me, and, andrea, r, last, day, of, school]
1451235    [just, finished, watching, your, song, present...
1181413       [janfran, awww, i, can, t, wait, to, get, one]
517910     [is, indeed, a, rather, large, hoarder, of, pa...
Name: text_processed, Length: 20000, dtype: object

In [8]:
def stem_and_lemmatize(tokens):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    from nltk.stem import PorterStemmer

    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in tokens]
    
    from nltk.stem import WordNetLemmatizer 
    from nltk.corpus import wordnet

    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(w) for w in tokens]
    
    if stemmed == lemmatized:
        return stemmed
    else: 
        return lemmatized

In [9]:
subset['text_processed'] = subset['text_processed'].apply(stem_and_lemmatize)

In [10]:
def remove_stopwords(tokens):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    from nltk.corpus import stopwords

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if not w in stop_words]
    filtered_tokens = [w for w in filtered_tokens if w]
    
    return filtered_tokens

In [11]:
subset['text_processed'] = subset['text_processed'].apply(remove_stopwords)

In [12]:
subset

,target,id,date,flag,user,text,text_processed
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,"[chrishasboobs, ahhh, hope, ok]"
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...","[misstoriblack, cool, tweet, apps, razr]"
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,"[tiannachaos, know, family, drama, lame, hey, ..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,"[school, email, open, geography, stuff, revise..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,"[upper, airway, problem]"
...,...,...,...,...,...,...,...
1178000,4,1981517014,Sun May 31 09:19:12 PDT 2009,NO_QUERY,gemmaridlington,Loving the weather Although we're not having ...,"[loving, weather, although, bbq, evening, appa..."
667014,0,2245469775,Fri Jun 19 16:10:38 PDT 2009,NO_QUERY,musicroxmysockz,Me and andrea r. last day of school..... http...,"[andrea, r, last, day, school]"
1451235,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...,"[finished, watching, song, present, boystown]"
1181413,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one,"[janfran, awww, wait, get, one]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [13]:
# all_words = []

# for w in subset['text_processed']:
#     all_words.append(w)

In [14]:
# all_words = nltk.FreqDist(all_words)

In [15]:
from nltk import FreqDist

# combine all words in text_processed into a single list
all_words = [word for sublist in subset['text_processed'] for word in sublist]

# calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

#Create a list of the top 5,000 words sorted by frequency.
top_words = sorted(freq_dist, key=freq_dist.get, reverse=True)[:5000]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [16]:
#Create an empty matrix with the same number of rows as the number of tweets and the same number of columns as the number of top words.
num_tweets = len(subset['text_processed'])
num_top_words = len(top_words)
matrix = np.empty((num_tweets, num_top_words), dtype=bool)

#Loop through each tweet and each top word, and set the corresponding element in the matrix to True if the word is contained in the tweet, and False otherwise.
for i, tweet in enumerate(subset['text_processed']):
    for j, word in enumerate(top_words):
        is_word_present = word in tweet
        matrix[i, j] = is_word_present

In [17]:
len(matrix[0])

5000

In [18]:
unique, counts = np.unique(matrix[0], return_counts=True)

dict(zip(unique, counts))

{False: 4997, True: 3}

In [19]:
# Function that takes a row of the matrix and returns True if there are at least 4 True values, False otherwise.
def get_label(row):
    # Sum the values in the row and return True if the sum is 4 or greater, False otherwise.
    return sum(row) >= 4

# List to store the dictionaries.
feature_set = []

# Loop through each row of the matrix and create a dictionary.
for i in range(num_tweets):
    # Create a dictionary with the presence or absence of each top word in the current tweet.
    word_dict = {top_words[j]: matrix[i, j] for j in range(num_top_words)}
    
    # Determine the label for the current tweet based on the number of top words that appear in it.
    label = get_label(matrix[i])
    
    # Append the dictionary and label as a tuple to the feature_set list.
    feature_set.append((word_dict, label))

In [20]:
# True or false for is_positive (4 or more True coincidences)
for i in range(10):
    print(feature_set[i][1], end=' ')

False False True True False True True True True True 

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [21]:
# your code here

In [22]:
# Split into training and testing sets
training_set = feature_set[:10000]
testing_set = feature_set[10000:]

In [26]:
import nltk
# Training classifer
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [27]:
# 25 features most informative
classifier.show_most_informative_features(25)

Most Informative Features
                  though = True             True : False  =     21.9 : 1.0
                     gon = True             True : False  =     18.8 : 1.0
                     try = True             True : False  =     18.7 : 1.0
                      ur = True             True : False  =     16.0 : 1.0
                    next = True             True : False  =     14.4 : 1.0
                     tho = True             True : False  =     14.1 : 1.0
                      ta = True             True : False  =     13.0 : 1.0
                    even = True             True : False  =     11.8 : 1.0
                    read = True             True : False  =     11.6 : 1.0
                     big = True             True : False  =     11.1 : 1.0
                     amp = True             True : False  =     10.7 : 1.0
                  almost = True             True : False  =     10.5 : 1.0
               yesterday = True             True : False  =     10.5 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [28]:
print("Naive Bayes Algo Accuracy Percent: ", (nltk.classify.accuracy(classifier, testing_set)))

Naive Bayes Algo Accuracy Percent:  0.8977


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here